# Box Office Bomb Data Pipeline Assignment

**Total Marks: 20**

This notebook implements an end-to-end pipeline to:
1. Scrape box office bomb data from Wikipedia
2. Validate & clean data using Pydantic
3. Enrich with OMDb API data
4. Perform consistency checks
5. Create final categorized dataset

## Setup and Imports

In [ ]:
# Import required libraries
import pandas as pd
import requests
from bs4 import BeautifulSoup
from pydantic import BaseModel, field_validator, ValidationError
from typing import Optional
import re
import time
from pathlib import Path

## Task 1: Scrape the "Bombs" Table (4 Marks)

Extract raw data from the Wikipedia HTML file:
- Film Title (with symbols like § and †)
- Year
- Net production budget (may contain ranges like "$100–160")
- Estimated loss (Nominal column)

Note:
- You need to extract the entire raw string with the symbols, references, etc along with the titles.
- You must handle the nested headers in the Wikipedia table (Budget and Loss columns have sub-headers).

In [ ]:
def scrape_box_office_bombs():
    """
    Scrape the box office bombs table from the local HTML file.
    Returns a list of dictionaries with raw extracted data.
    """
    # TODO: Load the HTML file

    url = "https://en.wikipedia.org/wiki/List_of_biggest_box-office_bombs"
    headers = {
      "User-Agent": "MyWikiApp/1.0 (https://example.com; contact@example.com)"
    }
    try:
      response = requests.get(url, headers=headers) # sending get request to get HTML
      html = response.text # HTML response
    except Exception as e:
      print(f"Error Fetching URL: {e}")
      return

    # TODO: Parse with BeautifulSoup

    soup = BeautifulSoup(html, 'html.parser')

    # TODO: Find the main table - it's wikitable sortable with caption "Biggest box-office bombs"

    tables = soup.find_all('table', class_="wikitable") # find all table with class = wikitable
    table = None

    for tb in tables:
      caption_tag = tb.find('caption')
      if not caption_tag:
          continue

      # 2.)clean the text: strip whitespace
      caption_text = caption_tag.get_text(strip=True)

      # 3.)check if the phrase is INSIDE the text
      if "Biggest box-office bombs" in caption_text:
          table = tb
          break # Stop looping once you find it!

    if table is None:
      print("No Table with Caption 'Biggest box-office bombs' found")
      return

    raw_data = []

    # TODO: Iterate through rows (skip headers) and extract: 'raw_title', 'raw_year', 'raw_budget', 'raw_loss'.
    # Store extracted data in raw_data. Example entry: {'raw_title': 'Town & Country', 'raw_year': '2001', 'raw_budget': '$90', 'raw_loss': '$10.4'}

    rows = table.find_all('tr') # find all the table rows in table

    for row in rows:
      cells = row.find_all(['th', 'td']) ## all header or data cell from row

      raw_title = cells[0].get_text(strip=True) # title is the first column of table

      if raw_title == "Title" or raw_title == "Film" or raw_title == "Nominal": # skipping the headers of table
          continue

      try:
          raw_year = cells[1].get_text(strip=True) # year is second column of the table
          raw_budget = cells[2].get_text(strip=True) # budget is third column of the table

          raw_loss = cells[4].get_text(strip=True) # loss is fifth column of the table

          raw_data.append({ # appending the data in raw_data list
              "raw_title": raw_title,
              "raw_year": raw_year,
              "raw_budget": raw_budget,
              "raw_loss": raw_loss,
          }) # appending the data in list
      except Exception as e: ## error handling of scraping
            print(f"Error in Extracting Text: {e}")
            continue

    return raw_data

In [ ]:
# Test the scraping function
raw_movies = scrape_box_office_bombs()
print(f"Scraped {len(raw_movies)} movies")
print("\nLast 15 raw entries:")
for i, movie in enumerate(raw_movies[-15:]):
    print(f"\n{i+1}. {movie}")

Scraped 139 movies

Last 15 raw entries:

1. {'raw_title': 'Town & Country', 'raw_year': '2001', 'raw_budget': '$90', 'raw_loss': '$85'}

2. {'raw_title': 'Transformers: The Last Knight', 'raw_year': '2017', 'raw_budget': '$217–260', 'raw_loss': '$100+'}

3. {'raw_title': 'Treasure Planet', 'raw_year': '2002', 'raw_budget': '$140', 'raw_loss': '$85'}

4. {'raw_title': 'Tron: Ares†', 'raw_year': '2025', 'raw_budget': '$220', 'raw_loss': '$132.7'}

5. {'raw_title': 'Turning Red§', 'raw_year': '2022', 'raw_budget': '$175', 'raw_loss': '$173'}

6. {'raw_title': 'Valerian and the City of a Thousand Planets', 'raw_year': '2017', 'raw_budget': '$177.2–180', 'raw_loss': '$82'}

7. {'raw_title': 'West Side Story', 'raw_year': '2021', 'raw_budget': '$100', 'raw_loss': '$104'}

8. {'raw_title': 'Wild Wild West', 'raw_year': '1999', 'raw_budget': '$170', 'raw_loss': '$66.2'}

9. {'raw_title': 'Windtalkers', 'raw_year': '2002', 'raw_budget': '$115–120', 'raw_loss': '$76–81'}

10. {'raw_title': 'Wis

## Task 2: Pydantic Data Parsing & Validation (6 Marks)

Create a Pydantic model that:
- Cleans titles (removes §, †, and footnotes like [nb 2], [1])
- Parses numeric values (handles ranges, currency symbols)
- Validates year as integer

In [ ]:
class MovieData(BaseModel):
    """
    Pydantic model for validating and cleaning movie data.
    """
    # TODO: Define the 4 required fields with their types:
    # title (str), year (int), budget_millions (float), loss_millions (float)
    title: str
    year: int
    budget_millions: float | None = None
    loss_millions: float

    @field_validator('title', mode='before')
    @classmethod
    def clean_title(cls, v):
        """
        Remove footnote markers and special symbols from title.
        - Remove § (streaming symbol)
        - Remove † (currently playing symbol)
        - Remove footnotes like [nb 2], [1], etc.
        """
        # TODO: Implement title cleaning logic
        # Hint: Use .replace() for symbols and re.sub() for footnotes

        v = v.replace('§', '').replace('†', '') # replcaing with empty char

        v = re.sub(r'\[.*?\]', '', v) # matching the pattern which start with '[' and and with ']' and removing it

        return v.strip() # strip whitespace

    @field_validator('year', mode='before')
    @classmethod
    def validate_year(cls, v):
        """
        Ensure year is a valid integer.
        Remove any extra characters and convert to int.
        """
        # TODO: Clean string (remove non-digits) and convert to integer

        raw_year = re.search(r'\d{4}', str(v)) # Extract first 4 digit of year string

        if not raw_year:
          raise ValueError(f"Found Year with less that 4 digits") # total digits in year caanot be less than 4

        year = int(raw_year.group())

        if year < 1888 or year > 2030: # validate range
          raise ValueError(f"Year {year} is out of range")

        return year

    @field_validator('budget_millions', mode='before')
    @classmethod
    def parse_budget(cls, v):
        """
        Parse budget value:
        - Strip $ and other currency symbols
        - Handle ranges (e.g., "100–160") by calculating average
        - Remove reference tags
        """
        # TODO: Call your helper method _parse_numeric_value

        return MovieData._parse_numeric_value(v)

    @field_validator('loss_millions', mode='before')
    @classmethod
    def parse_loss(cls, v):
        """
        Parse loss value with same logic as budget.
        """
        # TODO: Call your helper method _parse_numeric_value
        return MovieData._parse_numeric_value(v)

    @staticmethod
    def _parse_numeric_value(v):
        """
        Helper method to parse numeric values with ranges.
        """
        # TODO: Implement logic to:
        # 1. Clean string (remove $, reference tags)
        # 2. Check for ranges (hyphen or en-dash) and calculate average
        # 3. Return a float

        if v is None:
            return None

        if isinstance(v, (int, float)):
          return float(v)

        v = re.sub(r'[^\d\.\–\-]', '', str(v)) # keep digits and  hyphens/en-dashes, remove all other

        parts = re.split(r'[–-]', v) # split if it containes hyphens/en-dashes

        values = [float(val) for val in parts if parts] # converting values of part in float

        if not values:
          return None # if no part, return None

        return sum(values)/len(values) # taking average

In [ ]:
# Validate and clean the raw data
validated_movies = []
failed_validations = []

for raw_movie in raw_movies:
    try:
        movie = MovieData( # make MovieData Pydantic object
            title=raw_movie['raw_title'],
            year=raw_movie['raw_year'],
            budget_millions=raw_movie['raw_budget'],
            loss_millions=raw_movie['raw_loss']
        )
        validated_movies.append(movie)
    except ValidationError as e: # error handling
        failed_validations.append({
            'raw_data': raw_movie,
            'error': str(e)
        })
        print(f"Validation failed for {raw_movie['raw_title']}: {e}")

print(f"\n{'='*60}")
print(f"Validation Results:")
print(f"Successfully validated: {len(validated_movies)} movies")
print(f"Failed validations: {len(failed_validations)} movies")
print(f"{'='*60}")

# Show first 3 validated movies
print("\nLast 15 validated movies:")
for i, movie in enumerate(validated_movies[-15:]):
    print(f"\n{i+1}. {movie.model_dump()}")


Validation Results:
Successfully validated: 139 movies
Failed validations: 0 movies

Last 15 validated movies:

1. {'title': 'Town & Country', 'year': 2001, 'budget_millions': 90.0, 'loss_millions': 85.0}

2. {'title': 'Transformers: The Last Knight', 'year': 2017, 'budget_millions': 238.5, 'loss_millions': 100.0}

3. {'title': 'Treasure Planet', 'year': 2002, 'budget_millions': 140.0, 'loss_millions': 85.0}

4. {'title': 'Tron: Ares', 'year': 2025, 'budget_millions': 220.0, 'loss_millions': 132.7}

5. {'title': 'Turning Red', 'year': 2022, 'budget_millions': 175.0, 'loss_millions': 173.0}

6. {'title': 'Valerian and the City of a Thousand Planets', 'year': 2017, 'budget_millions': 178.6, 'loss_millions': 82.0}

7. {'title': 'West Side Story', 'year': 2021, 'budget_millions': 100.0, 'loss_millions': 104.0}

8. {'title': 'Wild Wild West', 'year': 1999, 'budget_millions': 170.0, 'loss_millions': 66.2}

9. {'title': 'Windtalkers', 'year': 2002, 'budget_millions': 117.5, 'loss_millions': 

## Task 3: Enrich with OMDb Data (4 Marks)

Query OMDb API for each movie to get:
- Plot
- Metascore
- IMDb Rating
- Director
- Language

Handle API failures (Response='False') or missing fields ('N/A') gracefully by storing them as None. Do not delete the row.

In [ ]:
# OMDb API configuration
# NOTE: You need to get a free API key from http://www.omdbapi.com/apikey.aspx
OMDB_API_KEY = "4c3965c5"  # Replace with your actual API key
OMDB_BASE_URL = "http://www.omdbapi.com/"

def query_omdb(title: str, year: int) -> dict:
    """
    Query OMDb API for movie metadata.
    Returns dict with plot, metascore, imdb_rating, director, language, omdb_year.
    Returns None values if movie not found or API fails.
    """
    # TODO: Construct params and make GET request
    try:
      params = {
          "apikey": OMDB_API_KEY,
          "t": title,
          "type": "movie",
          "y": year
      } # parametes to pass in request

      response = requests.get(OMDB_BASE_URL, params=params)

      if response.ok:
        data = response.json()
      else:
        return None

      # TODO: Handle 'Response': 'False'
      if data.get("Response") == "False": # if response id false return None
        print(f"Movie not found in OMDb: {title} ({year})")
        return None

      # TODO: Extract fields and handle 'N/A' conversion to None/Numbers

      def parse(key: str, type=str): # parsing function to remove N/A value and convert to required type
        value = data.get(key, "N/A")
        if value == "N/A":
          return None
        try:
          return type(value)
        except:
          return None

      return {
          "plot": parse("Plot"),
          "metascore": parse("Metascore", int),
          "imdb_rating": parse("imdbRating", float),
          "director": parse("Director"),
          "language": parse("Language"),
          "omdb_year": parse("Year", int)
      }
    except Exception as e:
      print(f"Error Fetching Movie with title: {title}, {e}")
      return None

In [ ]:
# Enrich each validated movie with OMDb data
enriched_data = []

print("Querying OMDb API...")
# TODO: Loop through validated_movies, call query_omdb, and merge data
for movie in validated_movies:

  movie_dict = movie.model_dump() # make dictionary from pydantic model

  omdb_data = query_omdb(movie.title, movie.year) # call query function to get extra features

  if omdb_data:
    movie_dict.update(omdb_data) # update movie data dict based on response

  else:
    movie_dict.update({
        "plot": None,
        "metascore": None,
        "imdb_rating": None,
        "director": None,
        "language": None,
        "omdb_year": None,
    })

  enriched_data.append(movie_dict)
  time.sleep(0.1) # delay between requests to obey rate limiting

print(f"\nEnriched {len(enriched_data)} movies with OMDb data")

# Show sample enriched data
print("\nFirst enriched entry:")
print(enriched_data[0])

Querying OMDb API...
Movie not found in OMDb: The Nutcracker in 3D (2010)
Movie not found in OMDb: Snake Eyes: G.I. Joe Origins (2021)

Enriched 139 movies with OMDb data

First enriched entry:
{'title': 'The 13th Warrior', 'year': 1999, 'budget_millions': 130.0, 'loss_millions': 99.0, 'plot': 'A man, having fallen in love with the wrong woman, is sent by the sultan himself on a diplomatic mission to a distant land as an ambassador. Stopping at a Viking village port to restock on supplies, he finds himself unwittingly em...', 'metascore': 42, 'imdb_rating': 6.6, 'director': 'John McTiernan', 'language': 'English, Latin, Swedish, Norse, Old, Danish, Arabic', 'omdb_year': 1999}


## Task 4: Data Consistency Check (2 Marks)

Compare Wikipedia year with OMDb year:
- "Verified": Years match (±1 year tolerance)
- "Mismatch": Years differ by >1
- "Not Found": OMDb returned no data

In [ ]:
def determine_match_status(wiki_year: int, omdb_year: Optional[int]) -> str:
    """
    Determine the match status between Wikipedia and OMDb years.
    Returns "Verified", "Mismatch", or "Not Found".
    """
    # TODO: Implement logic
    if omdb_year is None:
      return "Not Found"

    if abs(wiki_year - omdb_year) > 1:
      return "Mismatch"

    return "Verified"

# TODO: Apply this function to your enriched_data list and add match_status to each entry

for data in enriched_data:
  data["match_status"] = determine_match_status(data["year"], data["omdb_year"])

# Show match status distribution
df_temp = pd.DataFrame(enriched_data)
print("Match Status Distribution:")
print(df_temp['match_status'].value_counts())

# Show some mismatches if any
mismatches = df_temp[df_temp['match_status'] == 'Mismatch']
if len(mismatches) > 0:
    print(f"\nSample Mismatches (showing up to 5):")
    print(mismatches[['title', 'year', 'omdb_year', 'match_status']].head())

Match Status Distribution:
match_status
Verified     137
Not Found      2
Name: count, dtype: int64


## Task 5: Final Dataset & Categorization (4 Marks)

Create final DataFrame with:
- Loss_Category based on estimated loss:
  - "Catastrophic": Loss ≥ \$100M

  - "Severe": Loss between \$50M and \$100M

  - "Moderate": Loss < \$50M
- Save to `box_office_failures.csv`

Required columns: Title, Year, Director, Language, Budget_Millions, Loss_Millions, Loss_Category, IMDb_Rating, Metascore, Match_Status

In [ ]:
def categorize_loss(loss_millions: float) -> str:
    """
    Categorize the financial loss into tiers.
    """
    # TODO: Implement categorization logic
    if loss_millions is None or pd.isna(loss_millions):
        return "Unknown"

    if loss_millions >= 100:
      return "Catastrophic"

    if loss_millions >= 50:
      return "Severe"

    return "Moderate"


# TODO: Create DataFrame from enriched_data

df_final = pd.DataFrame(enriched_data)

# TODO: Add Loss_Category

df_final['Loss_Category'] = df_final['loss_millions'].apply(lambda x: categorize_loss(x))

# TODO: Select and rename columns to match requirements

df_final = df_final.rename(columns={
    'loss_millions': 'Loss_Millions',
    'imdb_rating': 'IMDb_Rating',
    'metascore': 'Metascore',
    'budget_millions': 'Budget_Millions',
    'title': 'Title',
    'year': 'Year',
    'director': 'Director',
    'language': 'Language',
    'match_status': 'Match_Status'
    })

# Display summary statistics
print("Final Dataset Summary:")
print(f"Total movies: {len(df_final)}")
print(f"\nLoss Category Distribution:")
print(df_final['Loss_Category'].value_counts())
print(f"\nBasic Statistics:")
print(df_final[['Budget_Millions', 'Loss_Millions', 'IMDb_Rating', 'Metascore']].describe())

# Display first few rows
print(f"\nFirst 10 rows of final dataset:")
df_final.head(10)

Final Dataset Summary:
Total movies: 139

Loss Category Distribution:
Loss_Category
Severe          85
Catastrophic    46
Moderate         8
Name: count, dtype: int64

Basic Statistics:
       Budget_Millions  Loss_Millions  IMDb_Rating   Metascore
count       139.000000     139.000000   137.000000  134.000000
mean        129.552518      91.917266     5.816058   46.529851
std          58.126686      32.407538     1.098276   15.359693
min          17.000000      10.800000     2.100000    9.000000
25%          81.250000      70.550000     5.400000   37.000000
50%         117.500000      85.000000     6.000000   46.000000
75%         175.000000     108.500000     6.600000   56.750000
max         326.000000     218.000000     8.000000   85.000000

First 10 rows of final dataset:


,Title,Year,Budget_Millions,Loss_Millions,plot,Metascore,IMDb_Rating,Director,Language,omdb_year,Match_Status,Loss_Category
0,The 13th Warrior,1999,130.0,99.0,"A man, having fallen in love with the wrong wo...",42.0,6.6,John McTiernan,"English, Latin, Swedish, Norse, Old, Danish, A...",1999.0,Verified,Severe
1,The 355,2022,57.5,93.0,When a top-secret weapon falls into mercenary ...,40.0,5.6,Simon Kinberg,"English, Chinese, Spanish, French, German, Ara...",2022.0,Verified,Severe
2,47 Ronin,2013,200.0,96.0,A band of samurai sets out to avenge the death...,28.0,6.2,Carl Rinsch,"English, Japanese",2013.0,Verified,Severe
3,The Adventures of Baron Munchausen,1988,46.6,38.5,An account of Baron Munchausen's supposed trav...,69.0,7.1,Terry Gilliam,English,1988.0,Verified,Moderate
4,The Adventures of Pluto Nash,2002,100.0,96.0,"In the future, a man struggles to keep his lun...",12.0,3.9,Ron Underwood,English,2002.0,Verified,Severe
5,The Adventures of Rocky & Bullwinkle,2000,87.3,63.5,"When enemies Boris, Natasha, and Fearless Lead...",36.0,4.3,Des McAnuff,English,2000.0,Verified,Severe
6,The Alamo,2004,107.0,94.0,Based on the 1836 standoff between a group of ...,47.0,6.1,John Lee Hancock,"English, Spanish",2004.0,Verified,Severe
7,Alexander,2004,155.0,71.0,"Alexander, the King of Macedonia and one of th...",40.0,5.6,Oliver Stone,English,2004.0,Verified,Severe
8,Ali,2001,107.0,63.0,"A biography of sports legend Muhammad Ali, foc...",65.0,6.7,Michael Mann,"English, French, Swahili",2001.0,Verified,Severe
9,Allied,2016,85.0,82.5,"In 1942, a Canadian intelligence officer in No...",60.0,7.1,Robert Zemeckis,"English, French, German, Arabic",2016.0,Verified,Severe


In [ ]:
# Save to CSV
output_path = 'box_office_failures.csv'

# TODO: Save DataFrame to CSV

print(f"✓ Dataset saved to: {output_path}")
print(f"✓ Total records: {len(df_final)}")

✓ Dataset saved to: box_office_failures.csv
✓ Total records: 139


## Additional Analysis (Optional)

In [ ]:
# # Show some interesting insights
# print("Top 10 Biggest Box Office Bombs (by loss):")
# print(df_final.nlargest(10, 'Loss_Millions')[['Title', 'Year', 'Director', 'Loss_Millions', 'Loss_Category']])

# print("\n" + "="*60)
# print("\nMovies with Lowest IMDb Ratings:")
# lowest_rated = df_final[df_final['IMDb_Rating'].notna()].nsmallest(5, 'IMDb_Rating')
# print(lowest_rated[['Title', 'Year', 'Director', 'IMDb_Rating', 'Metascore', 'Loss_Millions']])

# print("\n" + "="*60)
# print("\nAverage Loss by Category:")
# print(df_final.groupby('Loss_Category')['Loss_Millions'].agg(['mean', 'count']))

# print("\n" + "="*60)
# print("\nTop 5 Most Common Directors in Box Office Bombs:")
# print(df_final['Director'].value_counts().head())

print(df_final.groupby('Director')['Loss_Millions'].agg(['sum']).idxmax())

sum    Jaume Collet-Serra
dtype: object
